
- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [2]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags

In [31]:
with open('data/AK_Juneau_1.json') as f:
   data = json.load(f)
data

{'status': 200,
 'data': {'total': 8,
  'count': 7,
  'results': [{'primary_photo': None,
    'last_update_date': None,
    'source': None,
    'tags': None,
    'permalink': '8477-Thunder-Mountain-Rd_Juneau_AK_99801_M94249-83842',
    'status': 'sold',
    'list_date': None,
    'open_houses': None,
    'description': {'year_built': None,
     'baths_3qtr': None,
     'sold_date': '2023-08-22',
     'sold_price': None,
     'baths_full': None,
     'name': None,
     'baths_half': None,
     'lot_sqft': None,
     'sqft': None,
     'baths': None,
     'sub_type': None,
     'baths_1qtr': None,
     'garage': None,
     'stories': None,
     'beds': None,
     'type': None},
    'branding': [{'name': None, 'photo': None, 'type': 'Office'}],
    'list_price': None,
    'lead_attributes': {'show_contact_an_agent': True},
    'property_id': '9424983842',
    'photos': None,
    'flags': {'is_new_construction': None,
     'is_for_rent': None,
     'is_subdivision': None,
     'is_continge

In [61]:
# Function to get nested values
def get_nested_value(data, *keys):
    try:
        for key in keys:
            data = data[key]
        return data
    except (TypeError, KeyError):
        return None

# Function to convert JSON to DataFrame
def json_to_dataframe(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    results = get_nested_value(data, 'data', 'results')
    if results is None:
        return pd.DataFrame()

    df = pd.json_normalize(results)
    return df

# Folder path containing JSON files
json_path = 'data'

# List of JSON files in the folder
json_files = [json for json in os.listdir(json_path) if json.endswith('.json')]

# Initialize an empty DataFrame to store the combined data
combined_data = pd.DataFrame()

# Iterate through each JSON file
for js in json_files:
    # Construct the full path to the JSON file
    json_file_path = os.path.join(json_path, js)

    # Apply the json_to_dataframe function to extract data from the current JSON file
    json_data = json_to_dataframe(json_file_path)

    # Concatenate the extracted data to the combined_data DataFrame
    combined_data = pd.concat([combined_data, json_data], ignore_index=True)

# Display the combined DataFrame
print(combined_data)

/var/folders/7y/xbwztrr130jff2n00m2fjrsc0000gn/T/ipykernel_64135/2205081171.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data = pd.concat([combined_data, json_data], ignore_index=True)
/var/folders/7y/xbwztrr130jff2n00m2fjrsc0000gn/T/ipykernel_64135/2205081171.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data = pd.concat([combined_data, json_data], ignore_index=True)
/var/folders/7y/xbwztrr130jff2n00m2fjrsc0000gn/T/ipykernel_64135/2205081171.py:40: FutureWarning

          last_update_date                                               tags  \
0     2024-01-17T00:01:55Z  [central_air, dishwasher, fireplace, forced_ai...   
1     2024-01-16T16:49:36Z  [disability_features, basement, garage_1_or_mo...   
2     2024-01-13T00:03:39Z  [carport, central_air, dishwasher, fireplace, ...   
3     2024-01-16T07:48:30Z  [central_air, dining_room, disability_features...   
4     2024-01-13T00:01:45Z           [dining_room, hardwood_floors, basement]   
...                    ...                                                ...   
8186                  None                                               None   
8187  2023-08-22T17:54:39Z  [hardwood_floors, single_story, garage_1_or_mo...   
8188  2023-08-18T19:18:03Z  [central_air, city_view, dining_room, family_r...   
8189  2023-08-17T19:23:27Z  [city_view, community_spa_or_hot_tub, communit...   
8190  2023-08-16T20:05:03Z  [central_air, community_security_features, den...   

                           

In [109]:
filter_words = ['state','city', 'tags', 'sold_price', 'property_id', 'built', 'sqft', 'baths^', 'beds', 'description.type']
filtered_columns = [col for col in combined_data.columns if any(word in col for word in filter_words)]
filtered_df = combined_data[filtered_columns]
filtered_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8191 entries, 0 to 8190
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   tags                         7638 non-null   object 
 1   property_id                  8159 non-null   object 
 2   description.year_built       7316 non-null   float64
 3   description.sold_price       6716 non-null   object 
 4   description.lot_sqft         6991 non-null   float64
 5   description.sqft             7323 non-null   float64
 6   description.beds             7504 non-null   float64
 7   description.type             8125 non-null   object 
 8   location.address.state       8159 non-null   object 
 9   location.address.city        8154 non-null   object 
 10  location.address.state_code  8159 non-null   object 
dtypes: float64(4), object(7)
memory usage: 704.0+ KB


At this point, ensure that you have all sales in a dataframe.
- Is each cell one value, or do some cells have lists?
- Maybe the "tags" will help create some features.
- What are the data types of each column?
- Some sales may not actually include the sale price.  These rows should be dropped.
- Some sales don't include the property type.
- There are a lot of None values.  Should these be dropped or replaced with something?

In [ ]:
# load and concatenate data here
# drop or replace values as necessary

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [121]:
# One hot encoding for the tags data
dummy_df = pd.get_dummies(filtered_df['tags'].explode()).groupby(level=0).sum()
ones_count_per_column = dummy_df.sum(axis=0)
threshold = 400
selected_columns = ones_count_per_column[ones_count_per_column >= threshold].index

# Concatenating the dummy data and the dataframe and removing tags column
combined_df = pd.concat([filtered_df, dummy_df[selected_columns]], axis=1)

# Dropping the 'tags' column
combined_df = combined_df.drop(columns=['tags'])
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8191 entries, 0 to 8190
Data columns (total 55 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   property_id                  8159 non-null   object 
 1   description.year_built       7316 non-null   float64
 2   description.sold_price       6716 non-null   object 
 3   description.lot_sqft         6991 non-null   float64
 4   description.sqft             7323 non-null   float64
 5   description.beds             7504 non-null   float64
 6   description.type             8125 non-null   object 
 7   location.address.state       8159 non-null   object 
 8   location.address.city        8154 non-null   object 
 9   location.address.state_code  8159 non-null   object 
 10  basement                     8191 non-null   int64  
 11  big_lot                      8191 non-null   int64  
 12  big_yard                     8191 non-null   int64  
 13  carport           

In [111]:
combined_df.head()

,tags,property_id,description.year_built,description.sold_price,description.lot_sqft,description.sqft,description.beds,description.type,location.address.state,location.address.city,...,views,volleyball,washer_dryer,water_view,waterfront,well_water,white_kitchen,wine_cellar,wooded_land,wrap_around_porch
0,"[central_air, dishwasher, fireplace, forced_ai...",3300959879,2001.0,345000,22651.0,1539.0,3.0,single_family,Pennsylvania,Harrisburg,...,0,0,1,0,0,0,0,0,0,0
1,"[disability_features, basement, garage_1_or_mo...",3149247078,1900.0,196000,2614.0,2429.0,3.0,multi_family,Pennsylvania,Harrisburg,...,0,0,0,0,0,0,0,0,0,0
2,"[carport, central_air, dishwasher, fireplace, ...",4686076173,1971.0,205000,13504.0,1120.0,3.0,single_family,Pennsylvania,Harrisburg,...,0,0,1,0,0,0,0,0,0,0
3,"[central_air, dining_room, disability_features...",4085014275,2013.0,295000,2688.0,2400.0,3.0,townhomes,Pennsylvania,Harrisburg,...,0,0,0,0,0,0,0,0,0,0
4,"[dining_room, hardwood_floors, basement]",3743385520,1900.0,106000,871.0,1478.0,3.0,multi_family,Pennsylvania,Harrisburg,...,0,0,0,0,0,0,0,0,0,0


- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE such as using central tendency?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- If you replace cities or states with numerical values, make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Drop columns that aren't needed.
- Don't keep the list price because it will be too close to the sale price.

In [ ]:
# perform train test split here
# do something with state and city
# drop any other not needed columns

**STRETCH**

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized

In [3]:
dataframe = pd.read_csv('finaldf.csv')
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8154 entries, 0 to 8153
Data columns (total 61 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   8154 non-null   int64  
 1   list_price                   7716 non-null   float64
 2   property_id                  8154 non-null   int64  
 3   price_reduced_amount         2484 non-null   float64
 4   description.year_built       7311 non-null   float64
 5   description.sold_price       6711 non-null   float64
 6   description.lot_sqft         6986 non-null   float64
 7   description.sqft             7318 non-null   float64
 8   description.stories          6255 non-null   float64
 9   description.beds             7499 non-null   float64
 10  description.type             8120 non-null   object 
 11  location.address.state       8154 non-null   object 
 12  location.address.state_code  8154 non-null   object 
 13  basement          

In [4]:
import datetime as dt

#Extracting the present year 
current_year = dt.datetime.now().year

#Calculating the age of property
dataframe['age'] = current_year - dataframe['description.year_built']
dataframe = dataframe.drop(columns=['description.year_built'])
dataframe.sample(10)

,Unnamed: 0,list_price,property_id,price_reduced_amount,description.sold_price,description.lot_sqft,description.sqft,description.stories,description.beds,description.type,...,trails,two_or_more_stories,updated_kitchen,view,views,washer_dryer,mean,median,std,age
6877,6877,NaN,9280609914,NaN,718800.0,NaN,NaN,NaN,NaN,condo,...,0,0,0,0,0,0,415662.878788,292000.0,2.643328e+05,NaN
3005,3005,575000.0,1560885732,NaN,NaN,10454.0,2664.0,2.0,4.0,single_family,...,0,1,0,0,0,0,NaN,NaN,NaN,24.0
3329,3329,NaN,9856795451,NaN,NaN,NaN,NaN,NaN,NaN,condo,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
5545,5545,60000.0,9950476472,NaN,NaN,NaN,NaN,NaN,3.0,mobile,...,0,0,0,0,0,0,449247.676471,94500.0,2.915843e+06,25.0
712,712,75000.0,8922976627,NaN,60000.0,13068.0,1722.0,1.0,3.0,single_family,...,0,0,0,0,0,0,186433.100000,159000.0,1.058255e+05,62.0
3499,3499,240000.0,9760054045,NaN,260000.0,NaN,416.0,3.0,1.0,condos,...,0,1,0,0,0,0,621292.707317,430000.0,6.176093e+05,196.0
4863,4863,249000.0,3812726004,NaN,245000.0,6534.0,1580.0,1.0,3.0,single_family,...,0,0,0,1,0,0,260701.190476,245000.0,1.390774e+05,23.0
3724,3724,169900.0,4188477710,NaN,178000.0,436.0,1050.0,NaN,2.0,condos,...,0,0,0,0,0,1,241920.053476,245000.0,7.128774e+04,35.0
1870,1870,850000.0,2419022040,NaN,NaN,958.0,960.0,1.0,1.0,condos,...,0,0,0,0,0,1,NaN,NaN,NaN,41.0
314,314,225000.0,1743528776,NaN,225000.0,NaN,778.0,3.0,1.0,condos,...,0,1,0,0,0,1,611149.095238,496000.0,3.433493e+05,9.0


In [15]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

#One-hot encode 'description.type' column
dataframe = pd.get_dummies(dataframe, columns=['description.type'])

#Drop the specified columns before converting to int
columns_to_drop = ['property_id', 'price_reduced_amount', 'description.stories', 'location.address.state', 'location.address.state_code', 'std']
dataframe = dataframe.drop(columns=columns_to_drop)

#Convert the one-hot encoded columns to int (assuming they are boolean)
#Replace 'one_hot_encoded_columns' with the actual list of columns created by get_dummies
one_hot_encoded_columns = dataframe.columns[dataframe.columns.str.startswith('description.type')]
dataframe[one_hot_encoded_columns] = dataframe[one_hot_encoded_columns].astype(int)

KeyError: "None of [Index(['description.type'], dtype='object')] are in the [columns]"

In [6]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8154 entries, 0 to 8153
Data columns (total 66 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Unnamed: 0                                    8154 non-null   int64  
 1   list_price                                    7716 non-null   float64
 2   description.sold_price                        6711 non-null   float64
 3   description.lot_sqft                          6986 non-null   float64
 4   description.sqft                              7318 non-null   float64
 5   description.beds                              7499 non-null   float64
 6   basement                                      8154 non-null   int64  
 7   big_lot                                       8154 non-null   int64  
 8   big_yard                                      8154 non-null   int64  
 9   carport                                       8154 non-null   i

In [7]:
dataframe.head()

,Unnamed: 0,list_price,description.sold_price,description.lot_sqft,description.sqft,description.beds,basement,big_lot,big_yard,carport,...,description.type_condo,description.type_condo_townhome_rowhome_coop,description.type_condos,description.type_duplex_triplex,description.type_land,description.type_mobile,description.type_multi_family,description.type_other,description.type_single_family,description.type_townhomes
0,0,334900.0,345000.0,22651.0,1539.0,3.0,1,1,1,0,...,0,0,0,0,0,0,0,0,1,0
1,1,185000.0,196000.0,2614.0,2429.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2,197000.0,205000.0,13504.0,1120.0,3.0,1,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,3,295000.0,295000.0,2688.0,2400.0,3.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,4,105000.0,106000.0,871.0,1478.0,3.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [13]:
#Cleaning the dataset

#Filling the missing sold_price data with listing price
dataframe['description.sold_price'].fillna(dataframe['list_price'], inplace=True)

#Removing rows with nulls/NAn
columns_to_check = ['age', 'description.sold_price', 'description.beds', 'description.lot_sqft', 'description.sqft', 'mean']
dataframe = dataframe.dropna(subset=columns_to_check)

#Turning the age of property into an integer
dataframe['age'] = dataframe['age'].astype(int)

#dropping list_price
dataframe = dataframe.drop(columns=['list_price'])

KeyError: 'list_price'

In [12]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6339 entries, 0 to 8153
Data columns (total 65 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Unnamed: 0                                    6339 non-null   int64  
 1   description.sold_price                        6339 non-null   float64
 2   description.lot_sqft                          6339 non-null   float64
 3   description.sqft                              6339 non-null   float64
 4   description.beds                              6339 non-null   float64
 5   basement                                      6339 non-null   int64  
 6   big_lot                                       6339 non-null   int64  
 7   big_yard                                      6339 non-null   int64  
 8   carport                                       6339 non-null   int64  
 9   central_air                                   6339 non-null   int64 